<a href="https://colab.research.google.com/github/tamiresco/Spike-Timing/blob/master/Timing_Part_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Introdução

* Data Frame 3: Propriedades das distribuiçoes (entropia, entropia conjunta, etc.)
* Feature Selection: teste de hipotese em cada feature
* Modelo: Com os atributos das distribuições da sessao 1 e 2(?) preveremos quando a porcentagem de acerto dos ratos na sessao 7(?)

Envirolment setting up

In [ ]:
# basicos
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import dask as dask
from scipy import stats

# google colab
from google.colab import files

# feature selection
!pip install boruta
from sklearn.ensemble import RandomForestClassifier
from boruta import BorutaPy
from sklearn.feature_selection import SelectKBest, f_classif

# model selection
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import r2_score, mean_absolute_error

# model
from xgboost import XGBRegressor

Medidas da Teoria da Informação que podem ser usadas


1. **[Entropy](https://en.wikipedia.org/wiki/Entropy_(information_theory))** is a basic quantity in information theory associated to any random variable, which can be interpreted as the average level of "information", "surprise", or "uncertainty" inherent in the variable's possible outcomes.
2. **[Joint entropy](https://en.wikipedia.org/wiki/Joint_entropy)** is a measure of the uncertainty associated with a set of variables.
3. **[Conditional entropy](https://en.wikipedia.org/wiki/Conditional_entropy)** quantifies the amount of information needed to describe the outcome of a random variable Y given that the value of another random variable X is known. Here, information is measured in shannons, nats, or hartleys. The entropy of Y conditioned on X is written as H(Y|X).
4. **[Cross entropy](https://en.wikipedia.org/wiki/Cross_entropy)** between two probability distributions p and q over the same underlying set of events measures the average number of bits needed to identify an event drawn from the set if a coding scheme used for the set is optimized for an estimated probability distribution q, rather than the true distribution p.
5. **[Kullback-Leibler divergence](https://en.wikipedia.org/wiki/Kullback–Leibler_divergence)** (also called relative entropy) is a measure of how one probability distribution is different from a second, reference probability distribution.
6. **Symmetrised Kullback-Leibler divergence**
7. **[Jensen-Shannon divergence](https://en.wikipedia.org/wiki/Jensen–Shannon_divergence)** is a method of measuring the similarity between two probability distributions.
8. **[Mutual information](https://en.wikipedia.org/wiki/Mutual_information)** of two random variables is a measure of the mutual dependence between the two variables. More specifically, it quantifies the "amount of information" (in units such as shannons, commonly called bits) obtained about one random variable through observing the other random variable. 
9. **Normalised mutual information** (7 variants)
10. **[Variation of information](https://en.wikipedia.org/wiki/Variation_of_information)**  the variation of information or shared information distance is a measure of the distance between two clusterings (partitions of elements).
11. **Lautum information**
12. **[Conditional mutual information](https://en.wikipedia.org/wiki/Conditional_mutual_information)** is, in its most basic form, the expected value of the mutual information of two random variables given the value of a third.
13. **[Interaction Information](https://en.wikipedia.org/wiki/Interaction_information)** is a generalization of mutual information, but unlike the mutual information, the interaction information can be either positive or negative. 
14. **Co-information** is the same as Interaction Information by wikipedia.
15. **[Multi-information](https://en.wikipedia.org/wiki/Total_correlation)** or total correlation (Watanabe 1960) is one of several generalizations of the mutual information.
16. **Binding information**
17. **[Residual entropy](https://en.wikipedia.org/wiki/Residual_entropy)** is the difference in entropy between a non-equilibrium state and crystal state of a substance close to absolute zero. 
18. **Exogenous local information**
19. **Enigmatic information**

###Funções
 

Feature Engenering


In [ ]:
def add_columns_mean(dataFrame2): 

  df_mean = dataFrame2.loc[:,['T', 'A', 'E']].applymap(lambda x: np.mean(x)).groupby(level=0).mean()
  df_mean.rename({'T': 'T_mean', 'A' : 'A_mean', 'E': 'E_mean'}, axis=1, inplace=True)
 
  df_std = dataFrame2.loc[:,['T', 'A', 'E']].applymap(lambda x: np.std(x)).groupby(level=0).mean() 
  df_std.rename({'T': 'T_std', 'A' : 'A_std', 'E': 'E_std'}, axis=1, inplace=True)
  
  df_kurtosis = dataFrame2.loc[:,['T', 'A', 'E']].applymap(lambda x: stats.kurtosis(x)).groupby(level=0).mean()
  df_kurtosis.rename({'T': 'T_kurtosis', 'A' : 'A_kurtosis', 'E': 'E_kurtosis'}, axis=1, inplace=True)

  dataFrame3 = pd.concat([df_mean, df_std, df_kurtosis], axis =1)

  return dataFrame3

def bays(likelihood, prior, posteriorTrue, tmax = 5):  # ou tmax = max(likelihood)

  # alisando as 3 curvas de distribuicao e colocando como maximo o tmax
  grid1, likelihood = kernelDensityEstimation (likelihood, dt=0.1, bw_method = 'scott', tmax=tmax)
  grid2, prior = kernelDensityEstimation (prior, dt=0.1, bw_method = 'scott', tmax=tmax)
  grid3, posteriorTrue = kernelDensityEstimation (posteriorTrue, dt=0.1, bw_method = 'scott', tmax=tmax)

  # formula de probalibilidade condicional
  posteriorEstimation = [prior[i] * likelihood[i] for i in range(len(likelihood))] 

  # comparacao distribuicao estimada por update baysiano e distribuicao real
  score = mean_absolute_error(posteriorEstimation, posteriorTrue)

  return (score) 

def add_columns_bays_error(dataFrame2, dataFrame3):

  # preparacao
  df = dataFrame2.drop(['ITI', 'B', 'C', 'D', 'F', 'G', 'H', 'I', 'J', 'success', 'fail', 'rate success' ], axis=1)
  score = []
  mean_score = []

  for j in df.columns:  
    for i in dataFrame3.index:

      # aplicando a funcao q compara a distribuicao posterior esperada com a real em 2 conjuntos
      score.append( bays(df.at[(i,1),j], df.at[(i,2),j], df.at[(i,3),j]))
      score.append( bays(df.at[(i,2),j], df.at[(i,3),j], df.at[(i,6),j]))

      # depois tira-se a media do erro dos dois conjuntos
      mean_score.append(np.mean(score))

      # esvazia para fazer o calculo para o proximo rato
      score = []

  # fragmentar o vetor por distribuição (cada distribuicao tem nRats q estao empilhalhos)
  nRats = len(dataFrame3.index) 
  dataFrame3['T_bays'] = mean_score[0 : nRats]
  dataFrame3['A_bays'] = mean_score[nRats : 2*nRats]
  dataFrame3['E_bays'] = mean_score[2*nRats : 3*nRats]

  return dataFrame3

In [ ]:
def kernelDensityEstimation (df, dt, bw_method, tmax = 0): 

  tmin = 0
  tmax = max(df) if tmax == 0 else tmax

  kde = stats.gaussian_kde(df, bw_method=bw_method ) # instanciar gaussian_kde
  counts = int((tmax - tmin) / dt) # criar intervalo com mesma quantidade de canais que o histograma  
  grid = np.linspace(tmin, tmax, counts) #  continuando a criação
  pdf = kde.evaluate(grid)
  
  return grid, pdf

def differentialEntropy(p, dt, unidade = 'digits'):
  
  # multiplicação por dt garante a continuidade da variavel
  if unidade == 'nats':
    h = -np.sum(p*np.log(p))*dt 
  elif unidade == 'digits':
    h = -np.sum(p*np.log10(p))*dt
  elif unidade == 'bits':
    h = -np.sum(p*np.log2(p))*dt

  return h

def entropy (df, dt = 0.1, bw_method = 'scott'): # bw é o parametro bandwidth que determina o quão suavisada sera a curva 

  grid, pdf = kernelDensityEstimation(df, dt, bw_method) # estimar densidade probabilistica com kernel 
  h_kde = differentialEntropy(pdf, dt)  # calcular sua entropia diferencial
  
  return np.round(h_kde,2)

def add_columns_entropy(dataFrame2, dataFrame3): #TO DO

  df = dataFrame2.drop([ 'success', 'fail', 'rate success' ], axis=1)
  df = df.applymap(lambda x: entropy(x)).groupby(level=0).mean()
  df.rename({'T': 'T_entropy', 'ITI': 'ITI_entropy', 'A' : 'A_entropy', 'B': 'B_entropy', 'C': 'C_entropy', 'D': 'D_entropy', 'E': 'E_entrpy', 'F': 'F_entropy', 'G': 'G_entropy', 'H': 'H_entropy', 'I': 'I_entropy', 'J': 'J_entropy'}, axis=1, inplace=True)
  dataFrame3 = pd.concat([dataFrame3, df], axis =1)

  return dataFrame3

Feature Selection


In [ ]:
def preprocessing (dataFrame2, dataFrame3): 

  X = dataFrame3.applymap(lambda x: 100*round(x,2)).astype('int32') #multiplicando por 100, arredondando e definindo como inteiro: 0.453 = 45 
  X = np.array(X) #transformando em np array pq o boruta pede isso

  sessions_drop = range(1, len(sessions)) # definindo q todas colunas menos a ultima serao eliminadas, assim a ultima sera a q o success rate vira o target
  y = dataFrame2.drop(columns=['T','ITI','A', 'B', 'C', 'D','E', 'F', 'G', 'H', 'I', 'J', 'success', 'fail']) # ficar so com o success rate
  y = y.apply(lambda x: 100*round(x,2)).astype('int32').drop(index=sessions_drop, level=1) 

  return X, y
  
def boruta (dataFrame2, dataFrame3):

  X, y = preprocessing(dataFrame2, dataFrame3)

  #iniciando funcoes 
  rfc = RandomForestClassifier(n_estimators=200, n_jobs=-1, class_weight= 'balanced_subsample' , max_depth=10) 
  boruta_selector = BorutaPy(rfc, n_estimators='auto', verbose=2, random_state=1, max_iter = 50, perc = 90) #TO DO entender melhor 

  #aplicando-as aos dados
  boruta_selector.fit(X, y)

  #colunas q passaram no teste de hipotese
  if (boruta.support_) != 0 :
    cols_support = X.columns[boruta.support_]
  if (boruta.upport_weak_) != 0 :
    cols_support_weak_ = X.columns[boruta.upport_weak_]

    return X[cols_support], X[cols_support_weak_]

### Rodando

Feature Engenering




In [ ]:
# lendo csv
dataFrame2 = pd.read_csv('dataFrame2_9.csv')

# arrumando index
dataFrame2.set_index(['rat', 'session'], inplace=True)

# pegando propriedades das distribuições
dataFrame3 = add_columns_mean(dataFrame2)
dataFrame3 = add_columns_bays_error(dataFrame2, dataFrame3)
dataFrame3 = add_columns_entropy(dataFrame2, dataFrame3)

# df.applymap(lambda x: max(x))

Feature Selection

In [ ]:
boruta (dataFrame2, dataFrame3)

### Model

Treino

In [ ]:
# separando os dados de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)

# treinando regressor
xgbr = xgb.XGBRegressor(verbosity=0)  # (?) (n_estimators=1000, learning_rate=0.05, n_jobs=4)
xgbr.fit(X_train, y_train) # (?) (X_train, y_train, early_stopping_rounds=5, eval_set=[(X_valid, y_valid)], verbose=False) 
score = xgbr.score(xtrain, ytrain)   
print("Training score: ", score) 

Validação

In [ ]:
# cross validataion com cv = 5 e cv = Kfold (para comparar)
scores = cross_val_score(X_train, y_train, cv=5)
print("Mean cross-validation score: %.2f" % scores.mean())

kfold = KFold(n_splits=10, shuffle=True)
kf_cv_scores = cross_val_score(xgbr, X_train, y_train, cv=kfold )
print("K-fold CV average score: %.2f" % kf_cv_scores.mean())

Teste

In [ ]:
# testando regressor
y_pred = xgbr.predict(X_test) # (?) nao deveria fazer com um X_valid ate eu ter certeza q é a ultima alteracao q estou fazendo no modelo
r2 = r2_score(y_test, y_pred)
print("r2: %.2f" % r2)
print("r2: %.2f" % (r2*(1/2.0)))

# graficos
x_ax = range(len(ytest))
plt.scatter(x_ax, y_test, s=5, color="blue", label="original")
plt.plot(x_ax, y_pred, lw=0.8, color="red", label="predicted")
plt.legend()
plt.show()

Rascunho

In [ ]:
  # def cross_validation (model, X, y, cv=5, n_jobs=4, scoring='r2'):

#   X, X_week = boruta(X, y)
#   score = cross_val_score(model, X, y, cv=cv, n_jobs=n_jobs, scoring=scoring)

#   print("Individuals scores: ", scores())
#   print("Average scores: ", scores.mean())